In [12]:
import joblib
import json

import pandas as pd

def loading_data(outliers = True):
    
    try:
        data = joblib.load("3_recommendations_for_9974_products.joblib")
    
        print("Successfuly imported necessary file for recommendation.")

    except:
        raise Exception("Could not load the necessary dictionary for making recommendations ! Please check the file.")
    
    if outliers:
        try:
            outliers = joblib.load("outliers.joblib")
        
            print("Outliers successfully loaded.")
        
        except:
            raise Exception("Outliers could not loaded successfully.")
            
    
    return data, outliers
        
class Recommender:
    
    def __init__(self, data, outliers, prod_list, n = 3):
        
        self.data = data
        self.outliers = outliers
        self.prod_list = prod_list
        self.n = n
        
    # Input Validation : : :

    def input_validation(self):
        
        cart = list(set(self.prod_list))
        
        for product in cart:
            
            if product not in self.data.keys():
                
                cart.remove(product)
                
                print(f"{product} is not included in the recommendation dataframe.", 
                      "Removed from the cart list for recommendations.")
                
            if product in self.outliers:
                
                print(f"{product} is in outliers list. There are no recommendations for them for now !")
                
        if len(cart) == 0:
            
            raise Exception("Cart is empty. Please add items for obtaining recommendations.")
            
        print("Input validation completed.")
            
        self.cart = cart
            

    def recommend(self):
    
        # recommender metodu yukarda bir input_validation yapıldığını assume ediyor !!! aldığı prod_list parametresi ordan
        # çıkmış / valide edilmiş olmalı...
        
        self.final_recoms = {}
        self.prods = []
        self.recoms = []
        self.scores = []
        self.index = []
        
        for i in range(self.n):
        
            for product in list(set(self.cart)):
            
                recommendations = self.data.get(product, "NA")
            
                recom = recommendations.get(i, "NA")
                score = recommendations.get(f"score_{i}", 0)
                
                self.index.append(i)
                self.prods.append(product)
                self.recoms.append(recom)
                self.scores.append(score)
                
            i += 1
            
        self.results = { "Index" : self.index, 
                    "Products" : self.prods,
                    "Recommendations": self.recoms,
                    "Scores" : self.scores
                      }
        
        df = pd.DataFrame(self.results).set_index("Index")
        
        # recommendationlar hali hazırda sepette bulunan ürünlerden farklı olsunlar !
        
        df = df[~df["Recommendations"].isin(list(set(self.cart)))]
        
        # LIFT SCORE a göre yukardan aşağıya sıralayalım.
        
        df.sort_values("Scores", ascending = False, inplace = True)
        
        self.final_recommendations = df.copy()
        
        df = df[:10][["Recommendations", "Scores"]]
        

        for index,row in df[:10][["Recommendations", "Scores"]].iterrows():
    
            self.final_recoms.update( { row["Recommendations"] : row["Scores"]  } )
        
        results_json = json.dumps(self.final_recoms, ensure_ascii = False)
        
        return results_json

In [6]:
# test1
cart_products_dict = {"item1": "İçim Ayran 1.5 Lt", 
   "item2" : "Ülker Bebe Bisküvisi 800 Gr", 
   "item3" :"Camsil Matik 750 ml", 
   "item4": "Torku Pratiko Baharatlı Tırtıklı Patates 1 kg", 
    "item5" :"Malta Erik 500 gr", 
    "item6": "İnegöl Köfte 500 gr", 
    "item7": "Limon Lamas 500 gr", 
    "item8" : "Şeftali Paket 500 gr"}

In [63]:
# test2 
cart_products_dict = {"item1": "Vernel Max Yumuşatıcı Konsantre Nergis-Lavanta 1440 Ml",
                      "item2" : "Reis Çiğköftelik Bulgur 1 kg",
                      "item3" : "Garnier Color Naturals 7/0 - Kumral Saç Boyası",
                      "item4" : "Lindt Swiss Tradition 145 g",
                      "item5" : "Pet Craft Kedi Tasması",
                      "item6" : "OGX Yenileyici Argan & Moro mlo Yağlı Şampuan 100 ml"}

In [31]:
# test3 
cart_products_dict = {"item1" : "İthal Muz 500 gr",
                      "item2" : "Filiz Makarna Spaghetti 500 Gr",
                      "item3" : "Silen Ultra Kağıt Havlu 12'li",
                      "item4" : "Magnum Mini Cookie & Badem 6'lı"}

In [39]:
# test4
cart_products_dict = {"item1" : "Activia Shot Sade 80 Ml",
                      "item2" : "Bal Küpü Toz Şeker 3 kg"}

In [47]:
# test5
cart_products_dict = {"item1" : "Kamp CSA 2 Kişilik Çadır Yeşil",
                      "item2" : "Red Tiger Istakoz Kuyruğu 80 gr",
                      "item3" : "SuperFresh Garnitür 570 gr",
                      "item4" : "Fairy Sıvı Bulaşık Deterjanı Limon 400 ml",
                      "item5" : "Ülker İçim Toz Puding Çilekli 125 g",
                      "item6" : "Hacı Şakir Elegan Güzellik Sabunu 4x70 g",
                      "item7" : "Vaseline Jel Krem Baby 100 ML",
                      "item8" : "Milupa Organik Tahıllı Karışık Meyve Püresi 200Gr",
                      "item9" : "Kotex Ultra Dörtlü Normal Hijyenik Ped 28'li",
                      "item10" : "Tivoli Bellezza Şef Bıçağı"}

In [64]:
cart_products = []

for key in cart_products_dict.keys():
    
    cart_products.append(cart_products_dict[key])

In [65]:
cart_products

['Vernel Max Yumuşatıcı Konsantre Nergis-Lavanta 1440 Ml',
 'Reis Çiğköftelik Bulgur 1 kg',
 'Garnier Color Naturals 7/0 - Kumral Saç Boyası',
 'Lindt Swiss Tradition 145 g',
 'Pet Craft Kedi Tasması',
 'OGX Yenileyici Argan & Moro mlo Yağlı Şampuan 100 ml']

In [66]:
data, outliers = loading_data(outliers = True)

Successfuly imported necessary file for recommendation.
Outliers successfully loaded.


In [67]:
recommender = Recommender(data = data, outliers = outliers, prod_list = cart_products, n = 3)

In [68]:
recommender.input_validation()

Input validation completed.


In [69]:
len(recommender.prod_list) , len(recommender.cart)

(6, 6)

In [70]:
recommender.recommend()

'{"Lındt Lındor Mılk Balls Cıkolata 200 gr": 54442.0, "OGX Orchid Oil Sülfatsız Şampuan 385 ml": 5444.2, "Evohe Evohe Lux Kedi Mama Kabı XL": 2016.3703703703702, "Bingo Renkli&Beyaz Toz Çamaşır Deterjanı 7 Kg": 1814.7333333333331, "Öncü Acı Biber Salçası Cam Kavanoz 700 gr": 1701.3125, "Raffaello T15 Çikolata 150 Gr": 1555.4857142857143, "Scotch Brite Ultra Konfor Tırnak Koruyucu Sünger 2\'li": 1183.5217391304348, "Whiskas Ördek 4x100 g Yavru Kedi Maması": 1008.1851851851851, "Boron Matik 4 Kg Toz Deterjan Beyazlar": 800.6176470588234, "Jungle Kedi Çimi Fileli": 672.1234567901234}'

In [71]:
recommender.final_recommendations

,Products,Recommendations,Scores
Index,,,
0,Lindt Swiss Tradition 145 g,Lındt Lındor Mılk Balls Cıkolata 200 gr,54442.000000
0,OGX Yenileyici Argan & Moro mlo Yağlı Şampuan ...,OGX Orchid Oil Sülfatsız Şampuan 385 ml,5444.200000
0,Pet Craft Kedi Tasması,Evohe Evohe Lux Kedi Mama Kabı XL,2016.370370
0,Vernel Max Yumuşatıcı Konsantre Nergis-Lavanta...,Bingo Renkli&Beyaz Toz Çamaşır Deterjanı 7 Kg,1814.733333
0,Reis Çiğköftelik Bulgur 1 kg,Öncü Acı Biber Salçası Cam Kavanoz 700 gr,1701.312500
1,Lindt Swiss Tradition 145 g,Raffaello T15 Çikolata 150 Gr,1555.485714
0,Garnier Color Naturals 7/0 - Kumral Saç Boyası,Scotch Brite Ultra Konfor Tırnak Koruyucu Süng...,1183.521739
1,Pet Craft Kedi Tasması,Whiskas Ördek 4x100 g Yavru Kedi Maması,1008.185185
1,Vernel Max Yumuşatıcı Konsantre Nergis-Lavanta...,Boron Matik 4 Kg Toz Deterjan Beyazlar,800.617647
